**Inclass material for day 4: SQL Query using `pandas`**

This notebook was made based on main materials `4_SQL_Query.ipynb`

Version: May 2023

# SQL Query using `pandas`

**Training Objectives**

- Querying from SQL Databases
- SQL Joins
- SQL Conditional Statements
- Flavors and Common Operators
- End to end data analysis

# Working with SQL Databases

## Database Connection
Sedikit berbeda dengan `pandas` yang semua operasinya dilakukan di komputer lokal Anda. Ketika Anda bekerja dengan SQL, kemungkinan besar Anda memiliki database relasional yang disimpan **terpusat** pada server dan dapat **diakses oleh beberapa client**. Berikut adalah ilustrasinya:

![](assets/clientserver.png)

Untuk menghubungkan client machine dengan server, kita perlu membuat sebuah connection melalui sebuah DBAPI. Pada Python, DBAPI ekuivalen dengan sebuah package yang memungkinkan seorang data analyst dapat membuat objek **database connection**, salah satunya adalah `sqlite3`. Berikut adalah ilustrasi dari DBAPI:

![](assets/dbapi.png)

Terdapat banyak package Python yang menyediakan fungsionalitas agar data analyst dapat bekerja dengan basis data (database). Berikut adalah contohnya:

<br>

<details>
    <summary>✨ Connecting to MySQL</summary>
    
```python
import pymysql
  
conn = pymysql.connect(
    host = HOST_NAME,
    port = PORT_NUMBER,
    user = USER_NAME,
    password = PASSWORD,
    db = DATABASE_NAME)
```
</details>

<br>

<details>
    <summary>✨ Connecting to Oracle</summary>
    
```python
import cx_Oracle
  
# data source name from tnsnames.ora file
dsn_tns = cx_Oracle.makedsn(
    HOST_NAME,
    PORT_NUMBER
    service_name = SERVICE_NAME)

# connection
conn = cx_Oracle.connect(
    user = USER_NAME,
    password = PASSWORD,
    dsn = dsn_tns)
```
</details>

<br>

<details>
    <summary>✨ Connecting to PostgreSQL</summary>
    
```python
import psycopg2

conn = psycopg2.connect(
    host = HOST_NAME,
    port = PORT_NUMBER,
    user = USER_NAME,
    password = PASSWORD,
    database = DATABASE_NAME)
```
</details>

<br>

<details>
    <summary>✨ Connecting to Microsoft SQL Server</summary>
    
```python
import pyodbc 
conn = pyodbc.connect(
    'Driver={ODBC Driver 17 for SQL Server};'
    'Server=host;'
    'PORT=1433;'
    'UID=user;'
    'PWD=password;'
    'Database=database;')
```
</details>

## Database Schema

Database `chinook.db` terdiri dari 11 tabel yang berisi sampel data dari sebuah toko yang menjual media digital:

- Data terkait media seperti pada tabel `tracks`, `albums`, `artists`, `genres`, `media_types`, `playlists`, `playlist_track` merupakan data asli dari library Apple iTunes.
- Informasi mengenai `customers` dan `employees` dibuat menggunakan nama dan alamat fiktif yang dapat ditemukan di Google maps, dan data lain yang diformat dengan baik (telepon, fax, email, dll).
- Informasi penjualan `invoices` dan `invoice_items` dihasilkan secara otomatis menggunakan data acak untuk periode tahun 2009-2013.

Skema berikut sering disebut sebagai **Entity Relationship Diagram (ERD)**, menunjukkan:
1. Entitas (Tabel)
2. Atribut (Kolom beserta tipe datanya)
3. Kardinalitas (Hubungan antar tabel)

![](assets/chinookschema2.png)

Gambar di atas adalah ERD untuk database chinook:

- Satu database terdiri dari beberapa tabel
- Setiap tabel terdiri dari beberapa atribut/kolom
- Terdapat kolom unik yang membedakan satu tabel dengan lainnya, disebut sebagai **Primary Key (PK)**. PK dapat digunakan sebagai kolom penghubung antar tabel saat dilakukan proses join (penggabungan) tabel.

Kemudian untuk membaca data kita menggunakan `pd.read_sql_query()` dan menyertakan connection yang telah dibuat:

```python
sales = pd.read_sql_query("SELECT * FROM sales", conn)
```

Saat melempar object `conn`, `pandas` menggunakan [SQLAlchemy](https://www.sqlalchemy.org/) sehingga setiap database dapat bekerja. Tenang saja, hal ini bukan sesuatu yang perlu Anda khawatirkan pada pembelajaran ini. Sebagai tahap awal, mari kita coba bagaimana mengkoneksikan Jupyter Notebook dengan database SQLite (menggunakan package `sqlite3`) yang disebut sebagai **connection**:

In [86]:
import sqlite3 # default package: Database API (DBAPI) / package
import pandas as pd

In [87]:
# membuat objek connection
conn = sqlite3.connect("data_input/chinook.db")

## `SELECT` Statements

`SELECT` digunakan untuk memilih **kolom** dari sebuah **tabel**.

Syntax `SELECT`:

```sql
SELECT <NAMA_KOLOM>
FROM <NAMA_TABLE>
```

Misal, saya hanya ingin mengambil kolom `AlbumId` dan `Title` dari tabel `albums`.

In [88]:
pd.read_sql_query('SELECT AlbumId, Title FROM albums', con = conn)

,AlbumId,Title
0,1,For Those About To Rock We Salute You
1,2,Balls to the Wall
2,3,Restless and Wild
3,4,Let There Be Rock
4,5,Big Ones
...,...,...
342,343,Respighi:Pines of Rome
343,344,Schubert: The Late String Quartets & String Qu...
344,345,Monteverdi: L'Orfeo
345,346,Mozart: Chamber Music


Gunakan command `SELECT *` untuk mengambil **semua kolom** dari tabel `albums`:

In [89]:
pd.read_sql_query('SELECT * FROM albums', con =conn)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
...,...,...,...
342,343,Respighi:Pines of Rome,226
343,344,Schubert: The Late String Quartets & String Qu...,272
344,345,Monteverdi: L'Orfeo,273
345,346,Mozart: Chamber Music,274


In [90]:
pd.read_sql_query('SELECT AlbumID, Title, ArtistId FROM albums',con = conn)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
...,...,...,...
342,343,Respighi:Pines of Rome,226
343,344,Schubert: The Late String Quartets & String Qu...,272
344,345,Monteverdi: L'Orfeo,273
345,346,Mozart: Chamber Music,274


💡 **Note**: Berbeda dari Python, perintah yang ada di SQL bersifat case **insensitive**. Query di atas dapat juga dituliskan sebagai `select * from albums`. Namun, demi kemudahan pembacaan, statement biasa dituliskan dengan huruf besar (dalam hal ini `SELECT` dan `FROM`).

In [91]:
pd.read_sql_query('select * from albums', con =conn)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
...,...,...,...
342,343,Respighi:Pines of Rome,226
343,344,Schubert: The Late String Quartets & String Qu...,272
344,345,Monteverdi: L'Orfeo,273
345,346,Mozart: Chamber Music,274


### ❓ Knowledge Check

We'll create a `DataFrame`: this time select all columns from the `artists` table. Recall that when we use `pd.read_sql_query()` command we pass in the SQL query as a string, and add a connection as the second parameter. Save the output as a `DataFrame`.

Your DataFrame should be constructed like this:

```python
__ = pd.read_sql_query("SELECT __ FROM __", conn)
```

**Question:** How many rows are there in your DataFrame?

In [92]:
# your code here
artist = pd.read_sql_query('SELECT * FROM artists', con = conn)

In [93]:
artist.shape

(275, 2)

In [94]:
pd.read_sql_query('SELECT * FROM artists', con = conn, index_col= 'ArtistId').head()

,Name
ArtistId,
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains


Kita akan sering menggunakan parameter berikut dalam method `pd.read_sql_query()`:

- `sql`: SQL query dalam bentuk string
- `con`: SQL connection
- `index_col`: nama atau index kolom yang ingin dijadikan index (seperti pada `pd.read_csv()`)
- `parse_dates`: nama kolom yang ingin dikonversi menjadi tipe data `datetime64` (seperti pada `pd.read_csv()`)

Silahkan kunjungi [official documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html) untuk detail lebih lanjut.

### LIMIT

Statement `LIMIT` digunakan untuk mengambil beberapa baris pertama pada data.

Contoh: Ambil **5 baris pertama** dari tabel `artists` lalu jadikan kolom `ArtistId` sebuah index dengan parameter `index_col`.

In [95]:
pd.read_sql_query('SELECT * FROM artists LIMIT 5', con = conn, index_col= 'ArtistId')

,Name
ArtistId,
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains


In [96]:
pd.read_sql_query('''
    SELECT * 
    FROM artists LIMIT 5
''', con = conn, index_col= 'ArtistId')

,Name
ArtistId,
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains


## SQL Joins

Statement `JOIN` digunakan untuk **menggabungkan dua tabel** menjadi satu tabel melalui **kolom penghubung** yang sama. Operasi `JOIN` dapat dilakukan lebih dari satu kali dalam satu SQL query. Berikut adalah diagram ilustrasi beberapa macam operasi SQL `JOIN`:

![](assets/sqljoins.png)
Credit: Data & Object Factory, LLC

- `LEFT JOIN` paling sering digunakan pada sebagian besar skenario bisnis. `LEFT JOIN` mengembalikan semua baris pada tabel kiri **terlepas** dari apakah ada baris yang cocok pada tabel kanan.
- `INNER JOIN` adalah tipe join yang sangat intuitif dan mudah dipahami. Query ini mengembalikan semua baris di tabel kiri yang cocok dengan tabel kanan.

**Note**:

- `RIGHT JOIN` hampir tidak pernah digunakan karena ekuivalen dengan `LEFT JOIN` hanya mengganti peletakkan tabel kiri dan kanannya saja.
- `FULL OUTER JOIN` sangat jarang digunakan. Selain alasan komputasi, tipe join ini mengembalikan semua baris dari kedua tabel terlepas dari apakah ada kecocokan atau tidak, sehingga menghasilkan DataFrame dengan banyak missing value.

Ilustrasi terkait ke-empat tipe join dapat dilihat pada [Visual JOIN](https://joins.spathon.com/).

Syntax `JOIN`:

```sql
SELECT <COLUMNS>
FROM <LEFT_TABLE>
[LEFT|INNER] JOIN <RIGHT_TABLE>
ON <LEFT_TABLE>.key = <RIGHT_TABLE>.key
```

Perhatikan kembali skema database `chinook.db`:

1. Tabel `albums`: 
    - `AlbumId`
    - `Title`
    - `ArtistId`


2. Tabel `artists`:
    - `ArtistId`
    - `Name` 
    
**Kasus:** Kita ingin mendapatkan DataFrame yang menampilkan semua baris dengan kolom `AlbumId`, `Title`, dan `Name`. Perhatikan bahwa kolom `Name` terdapat pada tabel `artists`, sedangkan kolom `AlbumId` dan `Title` terdapat pada tabel `albums`.

- Tabel kiri: `albums`
- Tabel kanan: `artists`

In [97]:
pd.read_sql_query('''
    SELECT AlbumId, Title, Name 
    FROM albums
    LEFT JOIN artists
    ON albums.ArtistId = artists.ArtistId
    LIMIT 5
''', con = conn).shape

(5, 3)

Perhatikan SQL query di atas:
- Tabel `albums` menjadi tabel kiri, sedangkan tabel `artists` menjadi tabel kanan
- Kedua tabel digabungkan menggunakan operasi `LEFT JOIN` melalui key `ArtistId`

Terkadang kita memiliki nama kolom ataupun tabel yang panjang dan redundan untuk diketik.

Penggunaan statement `AS` dapat digunakan untuk melakukan **aliasing** nama tabel dan nama kolom.

In [98]:
# re-create tabel di atas menggunakan aliasing
pd.read_sql_query('''
    SELECT a.AlbumId, a.Title, b.Name AS artist_name, c.Name
    FROM albums AS a
    LEFT JOIN artists AS b
    ON a.ArtistId = b.ArtistId
    LEFT JOIN tracks AS c
    ON a.AlbumId = c.AlbumId
    LIMIT 5
''', con = conn)

,AlbumId,Title,artist_name,Name
0,1,For Those About To Rock We Salute You,AC/DC,For Those About To Rock (We Salute You)
1,1,For Those About To Rock We Salute You,AC/DC,Put The Finger On You
2,1,For Those About To Rock We Salute You,AC/DC,Let's Get It Up
3,1,For Those About To Rock We Salute You,AC/DC,Inject The Venom
4,1,For Those About To Rock We Salute You,AC/DC,Snowballed


⚠️ Hati-hati! Saat melakukan `LEFT JOIN`, peletakkan tabel kiri dan kanan sangat berpengaruh. Sebagai perbanding, sekarang kita tukar tabel `artists` menjadi tabel kiri, sedangkan `albums` menjadi tabel kanan.

Tabel `x` di bawah akan menampilkan semua data `artists` walaupun artist tersebut tidak ada di tabel `albums`.

In [99]:
x = pd.read_sql_query(
    """
    SELECT AlbumId, Title, Name
    FROM artists
    LEFT JOIN albums
    ON artists.ArtistId = albums.ArtistId
    
    """, conn)

# cek baris dengan missing value
x[x.isna().any(axis=1)].shape

(71, 3)

### ❓ Knowledge Check

Create a `DataFrame` named `tracks`, containing all columns from the `tracks` table; Additionally, it should also contains:

- The `Title` column from the `albums` table
- The `Name` column from the `artists` table
- The `Name` column from the `genres` table

> **Hint 1**: In your `SELECT` statement, you can use `SELECT tracks.* FROM TRACKS` to select all columns from the `TRACKS` table
> 
> **Hint 2**: When we write `SELECT tracks.Name AS tracksName`, we are renaming the output column from `Name` to `tracksName` using a technique called column aliasing. You may optionally consider doing this for columns that share the same name across different tables 

- Set the `TrackId` column to be the index.
- Verify: the resulting `DataFrame` should has 3503 rows and 11 columns. 

<br>

<details>
    <summary><i>👉 Klik di sini untuk hint</i></summary>

> **Hint 1**: Pada kasus ini, Anda wajib menggunakan syntax `<nama_tabel>.<nama_kolom>` pada statement `SELECT`. Hal ini karena terdapat nama kolom yang sama dari tabel yang berbeda. Jangan lupa untuk menggunakan aliasing `AS` agar nama kolom akhir tidak ambigu.
    
> **Hint 2**: Anda dapat menggunakan `SELECT tracks.*` untuk mengambil semua kolom pada tabel `tracks`

> **Hint 3**: Anda dapat melanjutkan hasil operasi `LEFT JOIN` dengan `LEFT JOIN` berikutnya. Sebagai referensi, berikut adalah struktur kodenya:
    
```python
___ = pd.read_sql_query(
    """
    SELECT ___
    FROM ___
    LEFT JOIN ___
    ON ___
    LEFT JOIN ___
    ON ___
    LEFT JOIN ___
    ON ___
    """, conn, index_col=___)
```
</details>

In [100]:
# your code here


Perform EDA on `tracks` to answer the following question:

1. Use `tail()` to inspect the last 5 rows of data. Which genre is present in the last 5 rows of our `tracks` DataFrame (Check all that apply)?
    - [ ] Latin
    - [x] Classical
    - [x] Soundtrack
    - [ ] Pop

In [101]:
# your code here


2. Apply `pd.crosstab(..., columns='count')`, `.value_counts()`, or any other techniques you've learned to compute the frequency table of Genres in your DataFrame. Which is among the top 3 most represented genres in the `tracks` DataFrame?
    - [x] Latin
    - [ ] Classical
    - [ ] Soundtrack
    - [ ] Pop

In [102]:
# your code here


3. Use `groupby()` on Artist Name and compute the `mean()` on the `UnitPrice` of each tracks. You will realize that most artists price their tracks at 0.99 (`mean`) but there are several artists where the `mean()` is 1.99. Which of the Artist has a mean of 0.99 `UnitPrice`:
    - [ ] The Office
    - [ ] Aquaman
    - [x] Pearl Jam
    - [ ] Lost

In [103]:
# your code here


## SQL Aggregation

Di `pandas`, kita bisa menggunakan ketiga method berikut untuk membuat tabel agregasi:

- `.crosstab()`
- `.pivot_table()`
- `.groupby()`

Sedangkan di SQL, kita menggunakan statement `GROUP BY` yang diletakkan setelah `SELECT ... FROM ...`.

Misal kita ingin mengetahui top 5 `CustomerId` berdasarkan jumlah besaran transaksinya (`Total`) dan juga tampilkan banyak transaksinya:

- GROUP BY -> tabel aggregasi
- ORDER BY -> sorting tabel, secara default ASC (ascending, dari terkecil sampai terbesar)

In [104]:
top_cust = pd.read_sql_query("""

SELECT CustomerId, 

-- Menampilkan agregasi berupa SUM pada kolom total
SUM(Total) AS TotalValue, 

-- Menampilkan agregasi berupa COUNT pada kolom invoice id
COUNT(InvoiceId) AS Purchases 

FROM invoices

-- Menentukan kolom yang menjadi kelompok
GROUP BY CustomerId 

-- Menentukan kolom yang menjadi acuan untuk mengurutkan data
ORDER BY TotalValue DESC

-- Melimitasi data yang ditarik menjadi 5 baris teratas
LIMIT 5

""", conn, index_col='CustomerId')
top_cust

,TotalValue,Purchases
CustomerId,,
6,49.62,7
26,47.62,7
57,46.62,7
45,45.62,7
46,45.62,7


Perhatikan bagaimana query di atas dapat mengambil 5 customer teratas dari tabel `invoices`:

1. Data dikelompokkan berdasarkan `CustomerId` dengan statement `GROUP BY`
2. Terdapat dua fungsi agregasi yang digunakan: `SUM()` dan `COUNT()`, masing-masing mengagregasikan kolom `Total` dan `InvoiceId`. Perhatikan bahwa kolom yang diagregasi harus berupa data numerik dengan aggregate function: `SUM`,` AVG`, `COUNT`, ` MIN`, dan `MAX`.
3. Statement `ORDER BY` ditambahkan untuk mengurutkan tabel berdasarkan kolom `TotalValue` secara `DESC`.

💡 **Note**:

- SQL Statement `GROUP BY` ekuivalen dengan `.groupby()` pada `pandas`
- SQL Statement `ORDER BY` ekuivalen dengan `.sort_values()` pada `pandas`

### ❓ Knowledge Check

A music producer wants to produce a new song that is expected to explode in the market. Before starting to compose songs, he first conducted research on what genres of music were currently hits. As a data analyst, you are asked to assist the music producer in making the right decision.

**Hint:**

Edit the following code to find out the most popular `genres` from all invoice sales. Use different column to acquire the following information: Summation of `Total` sales, and number of tracks bought from the `Quantity` columns.

```python
top_genre = pd.read_sql_query(
    """
    SELECT genres.GenreId, genres.Name,
    _____(invoices.UnitPrice), _____(invoice_items.Quantity)
    FROM invoices
    LEFT JOIN _____ ON _____
    LEFT JOIN _____ ON _____
    LEFT JOIN _____ ON _____
    GROUP BY _____
    ORDER BY _____
    """,
    conn,
    index_col='GenreId'
)
```

**Question:** What are the top 5 genres that generated the most profit?

In [105]:
# your code here
pd.read_sql_query('''
    SELECT g.genreId , g.name AS genre_name,
    SUM(ii.UnitPrice) AS total_sales,
    SUM(ii.Quantity) AS total_quantity
    FROM invoices AS i
    
    LEFT JOIN invoice_items AS ii
    ON i.InvoiceId = ii.InvoiceID
    
    LEFT JOIN tracks AS t 
    ON ii.trackId = t.trackId
    
    LEFT JOIN genres AS g
    ON t.genreId = g.genreId
    
    GROUP BY g.GenreId
    ORDER BY total_sales DESC
    
    LIMIT 5
''', con = conn, index_col = 'GenreId')

,genre_name,total_sales,total_quantity
GenreId,,,
1,Rock,826.65,835
7,Latin,382.14,386
3,Metal,261.36,264
4,Alternative & Punk,241.56,244
19,TV Shows,93.53,47


In [106]:
pd.read_sql_query(''' SELECT * FROM Invoices as i
LEFT JOIN invoice_items AS ii
    ON i.InvoiceId = ii.InvoiceID
''', con = conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total,InvoiceLineId,TrackId,UnitPrice,Quantity
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,1,2,0.99,1
1,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,2,4,0.99,1
2,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,3,6,0.99,1
3,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,4,8,0.99,1
4,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,5,10,0.99,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,411,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86,2236,3136,0.99,1
2236,411,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86,2237,3145,0.99,1
2237,411,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86,2238,3154,0.99,1
2238,411,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86,2239,3163,0.99,1


💡 Panduan step-by-step:

1. Lakukan operasi `JOIN` tergantung dari kebutuhan kolom yang ingin dianalisis
2. Gunakan `GROUP BY` apabila ingin membuat tabel agregasi
3. Aplikasikan fungsi agregat di `SELECT`
4. Tambahkan kolom yang relevan untuk ditampilkan pada `SELECT`
5. Urutkan data menggunakan `ORDER BY`

## `WHERE` Statements

Sampai di sini, kita telah mempelajari beberapa statement SQL yang sering digunakan:

- `SELECT` statement
- SQL `JOIN`
- Aliasing
- SQL Aggregation dengan `GROUP BY`

Sekarang, kita akan melihat teknik untuk melakukan **conditional subsetting atau filter baris** menggunakan statement `WHERE` yang diikuti dengan **kondisi**.

### Logical Operator

- Kondisi `WHERE` dapat dikombinasikan dengan logical operator: `IS`, `AND`, `OR`, dan `NOT`:
    - `IS` sama saja seperti notasi matematis `=`
    - `IS NOT` seperti notasi `!=`
    
- Kondisi pada `WHERE` juga mendukung operator matematis seperti >, >=, <, dan <=

**Kasus**: kita ingin melakukan analisis terhadap semua data `invoices` yang terjadi di `BillingCountry` Germany, maka kita bisa menambahkan statement `WHERE` sebagai berikut:

Note:
- `df[df['BillingCountry'] == 'Germany']`: pada pandas kita menggunakan tanda sama dengan 2x, namun pada SQL hanya 1x
- Operator `=` sama saja seperti `IS`

In [107]:
# your code here
pd.read_sql_query('''
    SELECT * FROm invoices 
    WHERE BillingCountry = "Germany"
''', con  = conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,6,37,2009-01-19 00:00:00,Berger Straße 10,Frankfurt,None,Germany,60316,0.99
2,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
3,12,2,2009-02-11 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,13.86
4,29,36,2009-05-05 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98
5,30,38,2009-05-06 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,3.96
6,40,36,2009-06-15 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,13.86
7,52,38,2009-08-08 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,5.94
8,67,2,2009-10-12 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,8.91
9,95,36,2010-02-13 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,8.91


In [108]:
pd.read_sql_query('''
    SELECT * FROm invoices 
    WHERE BillingCountry IS "Germany"
''', con  = conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,6,37,2009-01-19 00:00:00,Berger Straße 10,Frankfurt,None,Germany,60316,0.99
2,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
3,12,2,2009-02-11 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,13.86
4,29,36,2009-05-05 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98
5,30,38,2009-05-06 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,3.96
6,40,36,2009-06-15 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,13.86
7,52,38,2009-08-08 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,5.94
8,67,2,2009-10-12 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,8.91
9,95,36,2010-02-13 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,8.91


Berlawanan dengan kasus di atas, misal kita ingin analisis semua data `invoices` dimana `BillingCountry` **selain** negara Germany:

- Notasi `!=` sama seperti `IS NOT`

In [109]:
# your code here
pd.read_sql_query('''
    SELECT * FROm invoices 
    WHERE BillingCountry != "Germany"
''', con  = conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
1,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
2,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
3,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
4,8,40,2009-02-01 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
...,...,...,...,...,...,...,...,...,...
379,408,25,2013-12-05 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,3.96
380,409,29,2013-12-06 00:00:00,796 Dundas Street West,Toronto,ON,Canada,M6J 1V1,5.94
381,410,35,2013-12-09 00:00:00,"Rua dos Campeões Europeus de Viena, 4350",Porto,None,Portugal,None,8.91
382,411,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86


In [110]:
pd.read_sql_query('''
    SELECT * FROm invoices 
    WHERE BillingCountry IS NOT "Germany"
''', con  = conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
1,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
2,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
3,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
4,8,40,2009-02-01 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
...,...,...,...,...,...,...,...,...,...
379,408,25,2013-12-05 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,3.96
380,409,29,2013-12-06 00:00:00,796 Dundas Street West,Toronto,ON,Canada,M6J 1V1,5.94
381,410,35,2013-12-09 00:00:00,"Rua dos Campeões Europeus de Viena, 4350",Porto,None,Portugal,None,8.91
382,411,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86


**Kasus**: Misalkan kita hanya tertarik untuk menganalisis data yang terjadi di Amerika (yaitu `BillingCountry` Canada dan USA). Kira-kira bagaimana cara mengimplementasikannya dalam bentuk query SQL?

In [111]:
# your code here
pd.read_sql_query('''
    SELECT * FROM invoices 
    WHERE BillingCountry = "USA" OR BillingCountry = "Canada" OR BillingCountry = "Germany"
''', con  = conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
2,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
3,6,37,2009-01-19 00:00:00,Berger Straße 10,Frankfurt,None,Germany,60316,0.99
4,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
...,...,...,...,...,...,...,...,...,...
170,405,20,2013-11-21 00:00:00,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,0.99
171,406,21,2013-12-04 00:00:00,801 W 4th Street,Reno,NV,USA,89503,1.98
172,407,23,2013-12-04 00:00:00,69 Salem Street,Boston,MA,USA,2113,1.98
173,408,25,2013-12-05 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,3.96


### `IN` Operator

Operator `IN` yang memungkinkan kita menentukan beberapa nilai untuk perbandingan. Misalnya, seperti pada kasus sebelumnya, kita ingin mengambil semua data `invoices` yang terjadi di `BillingCountry` Canada dan juga USA:

In [112]:
# your code here
pd.read_sql_query('''
    SELECT * FROM invoices 
    WHERE BillingCountry IN ("USA","Canada")
''', con  = conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
1,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
2,13,16,2009-02-19 00:00:00,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,0.99
3,14,17,2009-03-04 00:00:00,1 Microsoft Way,Redmond,WA,USA,98052-8300,1.98
4,15,19,2009-03-04 00:00:00,1 Infinite Loop,Cupertino,CA,USA,95014,1.98
...,...,...,...,...,...,...,...,...,...
142,405,20,2013-11-21 00:00:00,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,0.99
143,406,21,2013-12-04 00:00:00,801 W 4th Street,Reno,NV,USA,89503,1.98
144,407,23,2013-12-04 00:00:00,69 Salem Street,Boston,MA,USA,2113,1.98
145,408,25,2013-12-05 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,3.96


### ❓ Knowledge Check

Edit the following code to include a `WHERE` clause. We want the returned DataFrame to contain only the `Pop` genre and only when the `UnitPrice` of the track is 0.99:

```python
popmusic = pd.read_sql_query(
    """
    SELECT tracks.*, genres.Name AS GenreName
    FROM _____
    LEFT JOIN _____
    ON _____
    WHERE _____
    """,
    conn,
    index_col='TrackId'
)
```

**Question:** How many rows are there in `popmusic`?

In [113]:
# your code here

popmusic = pd.read_sql_query(
    """
    SELECT tracks.*, genres.Name AS GenreName
    FROM tracks 
    LEFT JOIN genres
    ON tracks.genreId = genres.genreId
    WHERE GenreName IS 'Pop' AND UnitPrice IS 0.99
    """,
    conn,
    index_col='TrackId' 
)

popmusic.shape

(48, 9)

Notes: ketika membandingkan sebuah kolom dengan sebuah nilai, pastikan nilai yang dibandingkan tipe datanya sama seperti pada di kolom.

### Operating Dates

Pada operasi kondisi pada statement `WHERE` sebelumnya, kita dapat mengambil semua data `invoices` pada negara Germany. Namun, juga umum untuk kita melakukan conditional query untuk mengambil data pada **rentang tanggal** tertentu.

Sebelum lanjut, mari kita lihat tipe data dari objek `germany` yang telah diperoleh sebelumnya:

In [114]:
germany = pd.read_sql_query("SELECT * FROM invoices WHERE BillingCountry = 'Germany'", conn)
germany.dtypes

InvoiceId              int64
CustomerId             int64
InvoiceDate           object
BillingAddress        object
BillingCity           object
BillingState          object
BillingCountry        object
BillingPostalCode     object
Total                float64
dtype: object

In [115]:
germany.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,6,37,2009-01-19 00:00:00,Berger Straße 10,Frankfurt,None,Germany,60316,0.99
2,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
3,12,2,2009-02-11 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,13.86
4,29,36,2009-05-05 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98


Perhatikan bahwa `InvoiceDate` dikenali sebagai tipe data `object`. **Method `pd.read_sql_query()` berperilaku seperti method `pd.read_csv()` dimana secara default membaca tipe data suatu kolom sebagai numerik dan objek.** Ini tidak berarti bahwa kolom tersebut disimpan menggunakan format string (umumnya dikenal sebagai `VARCHAR` dalam database SQL). Lihatlah skema tabel berikut:

In [116]:
invoices_table = pd.read_sql_query(
    """
    SELECT sql
    FROM sqlite_master
    WHERE name = 'invoices'
    """, conn)
print(invoices_table.loc[0,:].values[0])

CREATE TABLE "invoices"
(
    [InvoiceId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [CustomerId] INTEGER  NOT NULL,
    [InvoiceDate] DATETIME  NOT NULL,
    [BillingAddress] NVARCHAR(70),
    [BillingCity] NVARCHAR(40),
    [BillingState] NVARCHAR(40),
    [BillingCountry] NVARCHAR(40),
    [BillingPostalCode] NVARCHAR(10),
    [Total] NUMERIC(10,2)  NOT NULL,
    FOREIGN KEY ([CustomerId]) REFERENCES "customers" ([CustomerId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
)


Perlu diingat bahwa untuk DBMS yang berbeda-beda, maka cara mengambil skema tabel akan berbeda pula. Query di atas khusus untuk mengambil skema tabel dari database SQLite (`sqlite_master`).

Output dari query di atas termasuk dalam tipe **Data Definition Language (DDL)** yang digunakan untuk membuat tabel. Dengan membaca DDL, akan berguna untuk memahami skema tabel dari database sehingga kita dapat melakukan operasi yang sesuai. Pada skema tabel `invoices` terdapat informasi yang berguna seperti:

- `InvoiceId` sebagai primary key
- `InvoiceDate` disimpan sebagai tipe data `DATETIME` (format `YYYY-MM-DD HH:MM:SS`)
- `CustomerId` sebagai foreign key pada tabel `customers`

Jika Anda tidak disediakan dengan skema database dalam bentuk diagram, maka luangkan waktu untuk mempelajari setiap skema tabel melalui DDL-nya.

**Kasus**: Kita ingin meninjau penjualan (`invoices`) tahun lalu di seluruh negara pada tahun 2012 saja.

In [117]:
pd.read_sql_query("""
SELECT *
FROM invoices
WHERE invoicedate >= '2012-01-01' AND invoicedate <= '2012-12-31'
""", conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,250,55,2012-01-01 00:00:00,421 Bourke Street,Sidney,NSW,Australia,2010,13.86
1,251,10,2012-01-09 00:00:00,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,0.99
2,252,11,2012-01-22 00:00:00,"Av. Paulista, 2022",São Paulo,SP,Brazil,01310-200,1.98
3,253,13,2012-01-22 00:00:00,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,1.98
4,254,15,2012-01-23 00:00:00,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,3.96
...,...,...,...,...,...,...,...,...,...
78,328,15,2012-12-15 00:00:00,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,0.99
79,329,16,2012-12-28 00:00:00,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,1.98
80,330,18,2012-12-28 00:00:00,627 Broadway,New York,NY,USA,10012-2612,1.98
81,331,20,2012-12-29 00:00:00,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,3.96


### `BETWEEN` Operator

Melanjutkan kasus sebelumnya, apabila kita ingin menggunakan kondisi pada rentang tertentu, pendekatan yang umum digunakan adalah operator `BETWEEN`. Silahkan lengkapi code berikut dan lihat apakah data yang terambil sama seperti sebelumnya:

```python
invoice_2012 = pd.read_sql_query(
    """
    SELECT *
    FROM invoices
    WHERE _____ BETWEEN '_____' AND '_____'
    """,
    conn,
    parse_dates='InvoiceDate'
)

invoice_2012['InvoiceDate'].describe()
```

In [118]:
# your code here
invoice_2012 = pd.read_sql_query(
    """
    SELECT *
    FROM invoices
    WHERE InvoiceDate BETWEEN '2012-01-01' AND '2012-12-30'
    """,
    conn,
    parse_dates='InvoiceDate'
)

invoice_2012['InvoiceDate'].describe()

count                               82
mean     2012-07-02 10:49:45.365853696
min                2012-01-01 00:00:00
25%                2012-03-30 06:00:00
50%                2012-06-28 12:00:00
75%                2012-09-27 18:00:00
max                2012-12-29 00:00:00
Name: InvoiceDate, dtype: object

⚠️ **Hati-hati!**

Operator `BETWEEN` sejatinya adalah inclusive, di mana kondisi start dan endnya termasuk. Namun, saat kita membandingkan date (misal '2012-12-31') dengan datetime (pada kolom `InvoiceDate`), seakan-akan end tidak inclusive. Sebagai eksperimen, cobalah ganti kondisi pada statement `WHERE` menjadi:

```
WHERE InvoiceDate BETWEEN '2012-01-01' AND '2012-12-30'
```

Maka `InvoiceDate` pada tanggal `2012-12-30` tidak masuk.

Sebagai solusinya, lebih baik kita menambahkan komponen waktu (time) pada kondisi:
```
WHERE InvoiceDate BETWEEN '2012-01-01 00:00:00' AND '2012-12-30 00:00:00'
```

`BETWEEN 2 AND 5` mengembalikan nilai 2, 3, 4, 5

In [119]:
# bukti bahwa operator BETWEEN inclusive pada start dan end
pd.read_sql_query(
    """
    SELECT * FROM invoices
    WHERE InvoiceId BETWEEN 2 AND 5
    """,
    conn
)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
1,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
2,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
3,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


In [120]:
# MASALAH: ketika hanya menggunakan '2012-12-30' maka data tanggal 30 Desember 2012 TIDAK masuk
invoice_2012 = pd.read_sql_query(
    """
    SELECT *
    FROM invoices
    WHERE InvoiceDate BETWEEN '2012-01-01' AND '2012-12-30'
    """,
    conn,
    parse_dates='InvoiceDate'
)

invoice_2012['InvoiceDate'].describe()

count                               82
mean     2012-07-02 10:49:45.365853696
min                2012-01-01 00:00:00
25%                2012-03-30 06:00:00
50%                2012-06-28 12:00:00
75%                2012-09-27 18:00:00
max                2012-12-29 00:00:00
Name: InvoiceDate, dtype: object

In [121]:
# SOLUSI: masukkan komponen TIME (00:00:00) agar data transaksi pada tanggal 30 Desember 2012 juga termasuk
invoice_2012 = pd.read_sql_query(
    """
    SELECT *
    FROM invoices
    WHERE InvoiceDate BETWEEN '2012-01-01' AND '2012-12-30 00:00:00'
    """,
    conn,
    parse_dates='InvoiceDate'
)

invoice_2012['InvoiceDate'].describe()

count                               83
mean     2012-07-04 15:02:10.120482048
min                2012-01-01 00:00:00
25%                2012-03-31 12:00:00
50%                2012-06-30 00:00:00
75%                2012-09-29 12:00:00
max                2012-12-30 00:00:00
Name: InvoiceDate, dtype: object

### `LIKE` Operator

Operator `LIKE` sangat berguna jika kita perlu mencocokan bagian tertentu dari sebuah string daripada menggunakan operator sama dengan (`=`). `'107%'` yang Anda lihat dalam query ditujukan untuk mengekstrak nilai `BillingPostalCode` yang **dimulai** dengan angka 107. Ini sangat membantu ketika Anda ingin mengekstrak data hanya pada wilayah tertentu. Di negara Germany, kita akan tahu bahwa Wilmersdorf dan Tempelhof di Berlin memiliki kode pos dimulai dengan 107.

**Note:** Karakter `%` disebut juga sebagai [wildcard character](https://www.w3schools.com/sql/sql_wildcards.asp)

In [124]:
pd.read_sql_query(
    """
    SELECT * FROM invoices
    WHERE BillingCountry = 'Germany'
    AND BillingPostalCode LIKE '107%'
    """,
    conn
) 

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
1,29,36,2009-05-05 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98
2,30,38,2009-05-06 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,3.96
3,40,36,2009-06-15 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,13.86
4,52,38,2009-08-08 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,5.94
5,95,36,2010-02-13 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,8.91
6,104,38,2010-03-29 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,0.99
7,224,36,2011-09-20 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98
8,225,38,2011-09-20 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
9,236,38,2011-10-31 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,13.86


**Diskusi:**

- Jika `BillingPostalCode LIKE '107%'`, maka semua baris dengan kode pos **dimulai** angka 107 akan tampil.
- Jika `BillingPostalCode LIKE '%107'`, maka semua baris dengan kode pos **diakhiri** angka 107 akan tampil.

Menurut Anda apa yang akan muncul jika kita menggunakan `%` **sebelum dan sesudah** pencocokan pola?

%107%

- Untuk `%`: terserah boleh berapa karakterpun (0 sampai tak hingga)
- Untuk `_`: terserah tapi hanya 1 karakter

In [125]:
# code here
pd.read_sql_query(
    """
    SELECT * FROM invoices
    WHERE BillingCountry = 'Germany'
    AND BillingPostalCode LIKE '%107%'
    """,
    conn
) 

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
1,29,36,2009-05-05 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98
2,30,38,2009-05-06 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,3.96
3,40,36,2009-06-15 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,13.86
4,52,38,2009-08-08 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,5.94
5,95,36,2010-02-13 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,8.91
6,104,38,2010-03-29 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,0.99
7,224,36,2011-09-20 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98
8,225,38,2011-09-20 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
9,236,38,2011-10-31 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,13.86


In [128]:
# code here 
pd.read_sql_query(
    """
    SELECT * FROM invoices
    WHERE BillingCountry = 'Germany'
    AND BillingPostalCode LIKE '107__'
    """,
    conn
) 

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
1,29,36,2009-05-05 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98
2,30,38,2009-05-06 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,3.96
3,40,36,2009-06-15 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,13.86
4,52,38,2009-08-08 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,5.94
5,95,36,2010-02-13 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,8.91
6,104,38,2010-03-29 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,0.99
7,224,36,2011-09-20 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98
8,225,38,2011-09-20 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
9,236,38,2011-10-31 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,13.86


#### ❓ Knowledge Check

**Kasus**: Pada 5 data pertama `customerinv`, kita bisa melihat kolom `Company` mungkin tidak dapat diandalkan karena bernilai `None`. Tetapi jika Anda memperhatikan kolom `Email`, Anda dapat melihat beberapa customer memiliki domain email `apple`, yang bisa menjadi indikator perusahaan mereka.

In [129]:
customerinv = pd.read_sql_query(
    """
    SELECT FirstName, LastName, Email, Company,
    InvoiceId, InvoiceDate, BillingCountry, Total
    FROM invoices
    LEFT JOIN customers
    ON invoices.CustomerId = customers.CustomerId
    """,
    conn)

customerinv.head()

,FirstName,LastName,Email,Company,InvoiceId,InvoiceDate,BillingCountry,Total
0,Leonie,Köhler,leonekohler@surfeu.de,None,1,2009-01-01 00:00:00,Germany,1.98
1,Bjørn,Hansen,bjorn.hansen@yahoo.no,None,2,2009-01-02 00:00:00,Norway,3.96
2,Daan,Peeters,daan_peeters@apple.be,None,3,2009-01-03 00:00:00,Belgium,5.94
3,Mark,Philips,mphilips12@shaw.ca,Telus,4,2009-01-06 00:00:00,Canada,8.91
4,John,Gordon,johngordon22@yahoo.com,None,5,2009-01-11 00:00:00,USA,13.86


**Pertanyaan:** Bagaimana jika kita ingin menghitung jumlah pelanggan yang bekerja di Apple Inc. dengan menggunakan domain `Email` sebagai pengganti indikator `Company`?

Silahkan lengkapi kode berikut:

```python
applecust = pd.read_sql_query(
    """
    SELECT firstname, lastname, email, company,
    invoiceid, invoicedate, billingcountry, total
    FROM invoices
    LEFT JOIN customers
    ON invoices._____ = customers._____
    WHERE _____
    """,
    conn
)
```

Berdasarkan query tersebut, berapa pelanggan yang bekerja di Apple Inc.?

- [ ] 412
- [ ] 49
- [x] 7
- [ ] 14

In [138]:
# your code here
applecust = pd.read_sql_query(
    """
    SELECT firstname, lastname, email, company,
    invoiceid, invoicedate, billingcountry, total
    FROM invoices
    LEFT JOIN customers
    ON invoices.customerId = customers.customerId
    WHERE Email LIKE '%apple%'
    GROUP BY Email
    """,
    conn
)

applecust


,FirstName,LastName,Email,Company,InvoiceId,InvoiceDate,BillingCountry,Total
0,Astrid,Gruber,astrid.gruber@apple.at,None,78,2009-12-08 00:00:00,Austria,1.98
1,Daan,Peeters,daan_peeters@apple.be,None,3,2009-01-03 00:00:00,Belgium,5.94
2,Hugh,O'Reilly,hughoreilly@apple.ie,None,10,2009-02-03 00:00:00,Ireland,5.94
3,Isabelle,Mercier,isabelle_mercier@apple.fr,None,84,2010-01-08 00:00:00,France,1.98
4,Ladislav,Kovács,ladislav_kovacs@apple.hu,None,85,2010-01-08 00:00:00,Hungary,1.98
5,Terhi,Hämäläinen,terhi.hamalainen@apple.fi,None,53,2009-08-11 00:00:00,Finland,8.91
6,Tim,Goyer,tgoyer@apple.com,Apple Inc.,15,2009-03-04 00:00:00,USA,1.98


## 📝 Summary

### ❓ Knowledge Check 1: SQL Statement Function

Kita telah mempelajari banyak statement pada SQL, silahkan mencocokan statement sesuai dengan kegunaannya:

**KEGUNAAN**

A. Memberikan nama lain pada tabel maupun kolom 

B. Mengambil beberapa baris teratas dari tabel 

C. Mengurutkan baris berdasarkan nilai pada kolom 

D. Mengambil kolom dari sebuah tabel 

E. Menggabungkan dua tabel menjadi satu tabel berdasarkan kolom penghubung 

F. Filter baris 

G. Membuat tabel agregasi 

**STATEMENT**

1. `SELECT <nama_kolom> FROM <nama_tabel>` D

2. `LIMIT <banyaknya_baris>` B

3. `AS <nama_kolom_atau_tabel>` A 

4. `<tabel_kiri> [LEFT|INNER] JOIN <tabel_kanan> ON <tabel>.key = <tabel>.key` E

5. `GROUP BY <nama_kolom>` G

6. `ORDER BY <nama_kolom> [ASC|DESC]` C

7. `WHERE <kondisi>` F


### ❓ Knowledge Check 2: SQL Statement Structure

Susunlah 8 baris SQL statement berikut menjadi sebuah kerangka urutan syntax yang benar:

1. `GROUP BY <nama_kolom>`

2. `LIMIT <banyaknya_baris>` 

3. `SELECT <nama_kolom> AS ...` 

4. `[LEFT|INNER] JOIN <tabel_kanan> AS ...` 

5. `ORDER BY <nama_kolom> [ASC|DESC]` 

6. `FROM <nama_tabel> AS ...` 

7. `WHERE <kondisi>`
    
8. `ON <tabel>.key = <tabel>.key` 

> **JAWABAN**: Urutan yang benar adalah  

3 - 6 - 4 - 8 - 7 - 1 - 5 - 2

### 💭 Dive Deeper

Toko musik digital ini ingin memberikan satu penghargaan ke `employees` yang telah bekerja keras melakukan penjualan di `BillingCountry` Amerika (yaitu **USA dan Canada**). Apabila penghargaan tersebut diberikan kepada `employees` berdasarkan **jumlah `Total`** penjualan, siapakah yang berhak mendapatkannya? Tampilkan **nama lengkap (`FirstName` dan `LastName`) beserta total penjualannya**.

Langkah pengerjaan
1. Pilih tabel yang mana saja?
2. JOIN table
3. Filter baris menggunakan WHERE
4. buat tabel agregasi dengan GROUP BY -> jangan lupa aggfunc di SELECT
5. Pilih kolom yang relevan di statement SELECT

In [145]:
# your code here

pd.read_sql_query('''
    SELECT e.FirstName, e.LastName, I.BillingCountry, SUM(Total) AS Total_penjualan
    FROM Invoices AS i
    
    LEFT JOIN Customers AS c
    ON i.CustomerId = c.CustomerId
    LEFT JOIN Employees AS e
    ON c.SupportRepId = e.EmployeeId
    
    WHERE BillingCountry IN ("USA","Canada")
    
    GROUP BY EmployeeId
    ORDER By Total_penjualan DESC
''', con = conn)


,FirstName,LastName,BillingCountry,Total_penjualan
0,Jane,Peacock,USA,310.96
1,Margaret,Park,USA,277.34
2,Steve,Johnson,Canada,238.72


### SQL Subquery

Kita sudah belajar cara untuk mendapatkan top customer kita dengan melakukan agregasi pada data. Jika dari informasi tersebut, kita ingin mengambil seluruh invoice dari top customer kita, maka kita dapat menggunakan `WHERE` statement menggunakan `IN` operator dan memanfaatkan subquery untuk membuat kondisi.

In [146]:
top_cust = pd.read_sql_query("""
    SELECT c.CustomerId
    FROM Customers as c 
    LEFT JOIN invoices as i 
    on i.CustomerId = c.CustomerId 
    GROUP BY c.CustomerId 
    ORDER BY SUM(Total) DESC 
    LIMIT 10
""", conn)
top_cust

,CustomerId
0,6
1,26
2,57
3,45
4,46
5,28
6,24
7,37
8,7
9,25


In [147]:
customerinv = pd.read_sql_query("""SELECT invoices.*  
                                 FROM invoices 
                                 WHERE invoices.CustomerId IN ( 
                                 
                                     SELECT c.CustomerId FROM Customers as c 
                                     LEFT JOIN invoices as i on i.CustomerId = c.CustomerId 
                                     GROUP BY c.CustomerId 
                                     ORDER BY SUM(Total) DESC LIMIT 10
                                     
                                )""", conn)
customerinv.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,46,6,2009-07-11 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,8.91
1,175,6,2011-02-15 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,1.98
2,198,6,2011-05-20 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,3.96
3,220,6,2011-08-22 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,5.94
4,272,6,2012-04-11 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,0.99


# Under and Over Fetching

Di antara semua tools yang telah kita pelajari untuk menganalisis data, sekarang saatnya kita merenungkan yang mana yang lebih cocok untuk Anda. Untuk meninjau, mari kita mengingat kembali apa yang telah kita pelajari:

- Reading flat files (CSV file)
- Data cleansing and wrangling
- Exploratory data analysis tools
- Visual exploratory tools

Kira-kira SQL masuk ke bagian apa? Untuk menjawab hal tersebut, Anda perlu memahami arsitektur client-server.`

![](assets/clientserver.png)

Sedikit berbeda dengan `pandas` yang semua operasinya dilakukan di komputer lokal Anda. Ketika Anda bekerja dengan SQL, kemungkinan besar Anda memiliki database relasional yang disimpan terpusat pada server dan dapat diakses oleh beberapa client.

Saat Anda melakukan query, sebenarnya Anda sedang menjalankan perintah untuk mengunduh data ke komputer lokal. Proses pengunduhan ini membutuhkan sumber daya dan perlu memanfaatkan alat secara efektif untuk meminimalkan biaya.

- **Over** fetching adalah kondisi dimana kita menarik data dari database **lebih dari** yang dibutuhkan, sehingga membutuhkan biaya dan waktu yang lebih.

- **Under** fetching adalah kondisi sebaliknya, dimana data yang ditarik **kurang dari** yang dibutuhkan, sehingga proses analisis tidak dapat dilakukan secara lengkap.

**Diskusi:**

Anda diminta untuk melakukan analisis terhadap semua penjualan (`invoices`) genre `Rock` pada tahun 2012. Pertimbangkan pertanyaan berikut:

- Apakah perlu bagi Anda untuk mengunduh semua tabel `tracks` ke komputer lokal?
- Apakah Anda melakukan filter baris terhadap `tracks` dengan genre `Rock` menggunakan SQL statement `WHERE` atau conditional subsetting `pandas`?
- Karena kita memerlukan informasi dari beberapa tabel, manakah cara yang lebih nyaman: melakukan query dengan `JOIN` atau melakukan `SELECT` secara terpisah dari database?

Cobalah membuat query yang paling optimum menurut Anda:

In [156]:
# your code here
pd.read_sql_query('''
    SELECT *
    FROM Tracks as t
    
    LEFT JOIN Genres as g
    ON t.GenreId = g.GenreId
    LEFT JOIN invoice_items as ii
    ON t.trackId = ii.trackId
    LEFT JOIN Invoices as i
    ON ii.invoiceId = i.invoiceId
    
    WHERE invoiceDate BETWEEN "2012-01-01" AND "2012-12-31 00:00:00" 
    AND g.Name IS "Rock"
''', con = conn)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,InvoiceLineId,InvoiceId,Quantity,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,3,Rock,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99,1728,319,1,13,2012-11-01 00:00:00,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,8.91
1,9,Rock,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99,1729,319,1,13,2012-11-01 00:00:00,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,8.91
2,15,Rock,4,1,1,AC/DC,331180,10847611,0.99,1730,319,1,13,2012-11-01 00:00:00,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,8.91
3,21,Rock,4,1,1,AC/DC,254380,8331286,0.99,1731,319,1,13,2012-11-01 00:00:00,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,8.91
4,30,Rock,5,1,1,"Steven Tyler, Richie Supa",356519,11616195,0.99,1732,320,1,22,2012-11-06 00:00:00,120 S Orange Ave,Orlando,FL,USA,32801,13.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,3114,Rock,246,1,1,"Dave Kushner, Duff, Matt Sorum, Scott Weiland ...",248398,8118785,0.99,1661,306,1,5,2012-09-05 00:00:00,Klanova 9/506,Prague,None,Czech Republic,14700,16.86
160,3280,Rock,256,2,1,None,515435,8270194,0.99,1691,312,1,34,2012-10-01 00:00:00,Rua da Assunção 53,Lisbon,None,Portugal,None,10.91
161,3286,Rock,256,2,1,None,552308,8858616,0.99,1692,312,1,34,2012-10-01 00:00:00,Rua da Assunção 53,Lisbon,None,Portugal,None,10.91
162,3292,Rock,257,2,1,None,390674,6491444,0.99,1693,312,1,34,2012-10-01 00:00:00,Rua da Assunção 53,Lisbon,None,Portugal,None,10.91
